In [7]:
import numpy as np
import scipy
# import pandas
import torch
import utils
from utils import generate_state_space_Matern_23
from scipy import linalg
from utils import build_id_key_table
from model_Bayes_diffusion import Bayes_diffu_tensor
import tqdm

data_file = '../processed_data/beijing_15k.npy'
full_data = np.load(data_file, allow_pickle=True).item()

fold=0

# here should add one more data-loader class
data_dict = full_data['data'][fold]
data_dict['ndims'] = full_data['ndims']
data_dict['num_node'] = full_data['num_node']

data_dict['time_id_table'] = full_data['time_id_table']
data_dict['time_uni'] = full_data['time_uni']


hyper_dict={}

hyper_dict['epoch'] = 2
hyper_dict['ls'] = 0.5
hyper_dict['var'] = 0.1
hyper_dict['device'] = torch.device("cpu")
hyper_dict['R_U'] = 3 # dim of each node embedding
hyper_dict['c'] = 1.0 # diffusion rate
hyper_dict['a0']=1.0
hyper_dict['b0']=1.0


F,P_inf = utils.generate_state_space_Matern_23(data_dict,hyper_dict)

data_dict['F'] = F
data_dict['P_inf'] = P_inf

In [8]:
N_T = len(data_dict['time_uni'])

model = Bayes_diffu_tensor(data_dict,hyper_dict)

# T=2
# model.msg_update_U_llk_del(T)
# model.msg_update_U_llk(T)

# model.msg_update_U_trans_del(T)
# model.msg_update_U_trans(T,mode='forward')

# model.msg_update_U_trans(T,mode='backward')

for epoch in tqdm.tqdm(range(hyper_dict['epoch'])):
    # forward
    for T in range(N_T):
        model.msg_update_U_llk_del(T)
        model.msg_update_U_llk(T)

        model.msg_update_U_trans_del(T)

        if T<N_T-1:
            model.msg_update_U_trans_vec(T,mode='forward')
            # model.msg_update_U_trans(T,mode='forward')

    # backward 
    for T in reversed(range(N_T-1)):
        
        # model.msg_update_U_trans(T,mode='backward')
        model.msg_update_U_trans_vec(T,mode='backward')

        model.msg_update_U_llk_del(T)
        model.msg_update_U_llk(T)

        model.msg_update_U_trans_del(T)

100%|██████████| 2/2 [00:22<00:00, 11.10s/it]
